<a href="https://colab.research.google.com/github/sshetty78/Face-Mask-detection/blob/main/Evalsingletask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
os.chdir("/content/drive/MyDrive/ColabFolder/")
import BKNetStyle
from const import *
import time
import math
%load_ext autoreload
%autoreload 2
from copy import copy, deepcopy

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
''' PREPARE DATA '''
''' PREPARE DATA '''
#smile_train, smile_test = CNN2Head_input.getSmileImage()
#gender_train, gender_test = CNN2Head_input.getGenderImage()
#age_train, age_test = CNN2Head_input.getAgeImage()

NUM_CLASSES = 3

def normalise(x):
    return (x-128)/255.0

def load_mini_dataset(index):
    
    label_to_int = {"Mask":0,"Mask_Mouth_Chin":1,"Mask_Chin":2,"Mask_Nose_Mouth":3}
    int_to_label = {0:"Mask",1:"Mask_Mouth_Chin",2:"Mask_Chin",3:"Mask_Nose_Mouth"}


    filename_prefix = '/content/drive/MyDrive/ColabFolder/test_data/test_part'#/content/drive/MyDrive/GT/BigDataProject/train_part0.npy
    filename = filename_prefix + str(index) + ".npy"
                
    X = np.load(filename, allow_pickle=True)
  
    x_mini_data =[]
    y_mini_data=[]
    mask = []    
    mask_prim = 0.0
    mask_sec = 1.0

    for sample in X:
        
        norm_x_prim = normalise(sample[0]) # todo check output once

        label_int = sample[1]
        y_prim = [0] * NUM_CLASSES

        if int_to_label[label_int]=="Mask":
          x_mini_data.append(norm_x_prim)
          y_prim[label_int] = 1 # y_ = [1,0,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)
          
        else:
          x_mini_data.append(norm_x_prim)
          y_prim[1] = 1 # y_ = [0,1,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)

          # create a copy for x & y, add same x and new y for secondary task
          norm_x_sec = deepcopy(norm_x_prim)
          y_sec = [0] * NUM_CLASSES
          y_sec[label_int-1] = 1
          
          x_mini_data.append(norm_x_sec)
          y_mini_data.append(y_sec)

          mask.append(mask_sec)
          

    return x_mini_data, y_mini_data,mask


def create_random_mini_batches(X, Y, mask_mini, mini_batch_size = 32):
    
    m = len(X) #.shape[0] # number of training examples
    mini_batches = []
    
    #print("Shape of X =", X.shape)
    #print("Shape of Y =", Y.shape)
    
    #Reshaping to convert Y to a 2D array from a rank one array
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(mask_mini)
    #Y = Y.reshape(Y.shape[0], 1) #todo Samarth check y shape
    # Y = one_hot(Y, NUM_CLASSES)
    #print(Y)

    #Shuffle the data in each of the mini batch
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :]
    shuffled_Z = Z[permutation]
    
    n_mini_batches = math.ceil(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(n_mini_batches):
        
        start_pos = k * mini_batch_size
        end_pos = min(start_pos + mini_batch_size, m)
        
        mini_batch_X = shuffled_X[start_pos : end_pos, :]
        mini_batch_Y = shuffled_Y[start_pos : end_pos, :]
        mini_batch_Z = shuffled_Z[start_pos : end_pos]
        
        mini_batch = (mini_batch_X, mini_batch_Y,mini_batch_Z)
        mini_batches.append(mini_batch)

    return mini_batches


In [ ]:
import numpy as np

def one_hot(indices, num_classes):
    tmp = np.zeros((indices.shape[0], num_classes), dtype=np.float32)
    tmp[np.arange(indices.shape[0]), indices] = 1.0
    return tmp


a = np.array([1, 0, 1, 2])
# print(a)
a = a.reshape(a.shape[0], 1)
# print(a)
# print(a.squeeze())
one_hot(a.squeeze(), 3)

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
sess = tf.compat.v1.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y_, mask = BKNetStyle.Input()

y_prim_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
import BKNetStyle
from const import *
import os
prim_loss, l2_loss, loss = BKNetStyle.selective_loss(y_prim_conv, y_, mask)
# train_step = BKNetStyle.train_op(loss, global_step)

prim_mask = tf.get_collection('face_mask')[0]
# sec_mask = tf.get_collection('sec_mask')[0]
# age_mask = tf.get_collection('age_mask')[0]

y_prim = tf.get_collection('y_face')[0]
# y_sec = tf.get_collection('y_sec')[0]
# y_age = tf.get_collection('y_age')[0]

prim_correct_prediction = tf.equal(tf.argmax(y_prim_conv, 1), tf.argmax(y_prim, 1))
# sec_correct_prediction = tf.equal(tf.argmax(y_sec_conv, 1), tf.argmax(y_sec, 1))
# age_correct_prediction = tf.equal(tf.argmax(y_age_conv, 1), tf.argmax(y_age, 1))

prim_true_pred = tf.reduce_sum(tf.cast(prim_correct_prediction, dtype=tf.float32) * prim_mask)
# sec_true_pred = tf.reduce_sum(tf.cast(sec_correct_prediction, dtype=tf.float32) * sec_mask)
# age_true_pred = tf.reduce_sum(tf.cast(age_correct_prediction, dtype=tf.float32) * age_mask)


print('Restore model')
saver = tf.train.Saver()
saver.restore(sess, './sample_data/' + 'model.ckpt')
print('OK')


prim_nb_true_pred = 0

prim_nb_train = 0

# print("Learning rate: %f" % learning_rate.eval())
    #added by Samarth; lazy loading of dataset and training
n_mini_batches = 0
epoch_cost = 0
total_time_taken_to_load_dataset = 0
total_time_taken_to_create_mini_batches = 0            
tic = time.time()
#file_indices = get_shuffled_indices(1, 26) # todo change to total part files
file_indices = [0] #TODO - to remove

mini_batch_size = BATCH_SIZE
true_pos_count = 0
false_pos_count = 0
false_neg_count = 0
true_neg_count = 0
TP = tf.count_nonzero(tf.cast(tf.argmax(y_prim_conv, 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask, axis=0)
FP = tf.count_nonzero(tf.cast(tf.argmax(y_prim_conv, 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask, axis=0)
FN = tf.count_nonzero(tf.cast((tf.argmax(y_prim_conv, 1) - 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask, axis=0)
TN = tf.count_nonzero(tf.cast((tf.argmax(y_prim_conv, 1) - 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask, axis=0)


for j in file_indices:
    
    X_mini = None
    Y_mini = None

    lmd_tic = time.time()
    X_mini, Y_mini, mask_mini = load_mini_dataset(j) # todo Samarth add for mask - done

    lmd_toc = time.time()
    total_time_taken_to_load_dataset += (lmd_toc-lmd_tic)
    
    rmb_tic = time.time()
    mini_batches = None
    mini_batches = create_random_mini_batches(X_mini, Y_mini, mask_mini, mini_batch_size = mini_batch_size) # todo Samarth add for mask
    rmb_toc = time.time()
    total_time_taken_to_create_mini_batches += (rmb_toc-rmb_tic)

    for mini_batch in mini_batches:
        batch_img = mini_batch[0] 
        batch_label = mini_batch[1]
        batch_mask = mini_batch[2]

        # (batch_img, batch_label) = mini_batch
        curr_batch_size = batch_img.shape[0]
        # batch_mask = np.zeros(curr_batch_size) #todo comment Samarth - done

        for i in range(curr_batch_size):
            if batch_mask[i] == 0.0:
                prim_nb_train += 1
            # else:
                # if batch_mask[i] == 1.0:
                    # sec_nb_train += 1                        
                # else:
                #     age_nb_train += 1
        # print("get shape batch img 1")
        # print(tf.shape(batch_img))
        # batch_img = CNN2Head_input.augmentation(batch_img, 48)
        # print(tf.shape(batch_img))
        # print(tf.shape(batch_label))

        # ttl, pml, l2l, _ = sess.run([loss, prim_loss, l2_loss, train_step],
        #                                       feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
        #                                                 phase_train: True, keep_prob: 0.5})

        prim_nb_true_pred += sess.run(prim_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                  phase_train: False,
                                                                  keep_prob: 0.5})

        true_pos, false_pos, false_neg, true_neg = sess.run([TP, FP, FN, TN], feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                  phase_train: False,
                                                  keep_prob: 0.5})
        true_pos_count += true_pos
        false_pos_count += false_pos
        false_neg_count += false_neg
        true_neg_count += true_neg


prim_train_accuracy = prim_nb_true_pred * 1.0 / prim_nb_train

precision_class0 = true_pos_count / (true_pos_count + false_pos_count)
recall_class0 = true_pos_count / (true_pos_count + false_neg_count)
f1_class0 = 2 * precision_class0 * recall_class0 / (precision_class0 + recall_class0)
print(true_pos_count, false_pos_count, false_neg_count, true_neg_count)

precision_class1 = true_neg_count / (true_neg_count + false_neg_count)
recall_class1 = true_neg_count / (true_neg_count + false_pos_count)
f1_class1 = 2 * precision_class1 * recall_class1 / (precision_class1 + recall_class1)

print('Primary task train accuracy: ' + str(prim_train_accuracy * 100))
# print('Secondary task train accuracy: ' + str(sec_train_accuracy * 100))
print('Precision: ', precision_class0)
print('Recall: ', recall_class0)
print('F1 score: ', f1_class0)
print('Precision: ', precision_class1)
print('Recall: ', recall_class1)
print('F1 score: ', f1_class1)

# print('Prim task test accuracy: ' + str(prim_train_accuracy * 100))


Restore model
INFO:tensorflow:Restoring parameters from ./sample_data/model.ckpt
OK
483 25 10 482
Primary task train accuracy: 96.7
Precision:  0.9507874015748031
Recall:  0.9797160243407708
F1 score:  0.9650349650349651
Precision:  0.9796747967479674
Recall:  0.9506903353057199
F1 score:  0.964964964964965
